### 1. **Setup + Imports Cell**

In [ ]:
import os
import sys

# 🔁 Add your project root to Python path
project_root = os.path.abspath("..")  # or "../ProjectName" if nested deeper
if project_root not in sys.path:
    sys.path.insert(0, project_root)
# Now you can import modules from the root directory

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from src.dataloader import get_dataloaders
from src.models.resnet import get_model  # or use build_effnet if swapping
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")

### 2. **Load Latest Model**

In [ ]:
def get_latest_model(path="../models", pattern="*.pt"):
    files = glob.glob(os.path.join(path, pattern))
    if not files:
        raise FileNotFoundError(f"No .pt files found in: {os.path.abspath(path)}")
    return max(files, key=os.path.getctime)


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = get_latest_model()
model = get_model()
model.load_state_dict(torch.load(model_path))
model.eval().to(DEVICE)

### 3. **Inference + Accuracy on Test Set**

In [ ]:
_, _, test_loader = get_dataloaders()
correct, total = 0, 0

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        preds = model(x)
        preds = torch.argmax(preds, dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"🎯 Test Accuracy: {correct/total:.4f}")

### 4. **Confusion Matrix + Save to `reports/`**

In [ ]:
def generate_and_save_confusion_matrix(model, dataloader, class_names, save_path="../reports/confusion_matrix.png"):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(DEVICE)
            outputs = model(x)
            predicted = torch.argmax(outputs, dim=1)

            preds.extend(predicted.cpu().tolist())
            targets.extend(y.tolist())

    cm = confusion_matrix(targets, preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    # Ensure reports directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Plot and save the confusion matrix
    plt.figure(figsize=(8, 6))
    disp.plot(cmap="Blues", xticks_rotation=45, values_format='d')
    plt.title("📊 Confusion Matrix")
    plt.savefig(save_path)
    plt.show()

    print(f"✅ Saved confusion matrix to: {save_path}")

In [ ]:
# 1. Run predictions over test set
all_preds, all_labels = [], []
class_names = [
    "Bear", "Bird", "Cat", "Cow", "Deer", "Dog", "Dolphin",
    "Elephant", "Giraffe", "Horse", "Kangaroo", "Lion", "Panda",
    "Tiger", "Zebra"
]
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        preds = torch.argmax(out, dim=1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(y.cpu().tolist())

# 2. Compute metrics
precision, recall, f1, support = precision_recall_fscore_support(
    all_labels, all_preds, labels=range(len(class_names)), zero_division=0
)

# 3. Build dataframe
results_df = pd.DataFrame({
    "Class": class_names,
    "Images": support,
    "Precision": np.round(precision, 4),
    "Recall": np.round(recall, 4),
    "F1-Score": np.round(f1, 4)
})

# 4. Display it
print("📊 Per-Class Evaluation Metrics")
display(results_df)

# 5. Save to CSV
results_df.to_csv("../reports/per_class_metrics.csv", index=False)


In [ ]:
model.load_state_dict(torch.load(model_path))
model.to(DEVICE)
_, _, test_loader = get_dataloaders()

class_names = [
    "Bear", "Bird", "Cat", "Cow", "Deer", "Dog", "Dolphin",
    "Elephant", "Giraffe", "Horse", "Kangaroo", "Lion", "Panda",
    "Tiger", "Zebra"
]

generate_and_save_confusion_matrix(model, test_loader, class_names)

### 5. **Plot Metrics from MLflow**

In [ ]:
import mlflow
mlflow.set_tracking_uri("file:../mlruns")

In [ ]:
def plot_accuracy(train_acc, val_acc, save_path="../reports/final_accuracy_plot.png"):
    epochs = list(range(1, len(train_acc) + 1))

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, train_acc, label='Train Accuracy', marker='o', color='seagreen')
    plt.plot(epochs, val_acc, label='Validation Accuracy', marker='s', color='darkred')

    plt.title("📈 Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.xticks(epochs)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
 
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path)
    plt.show()
    print(f"✅ Accuracy plot saved to: {save_path}")


In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
experiment = client.get_experiment_by_name("animal_classifier_v2")
# run = sorted(client.search_runs(experiment.experiment_id), key=lambda r: r.start_time)[-1]
run = sorted(
    client.search_runs([experiment.experiment_id]),
    key=lambda r: r.info.start_time
)[-1]

run_id = run.info.run_id

train_acc = [m.value for m in client.get_metric_history(run_id, "train_acc")]
val_acc = [m.value for m in client.get_metric_history(run_id, "val_acc")]

plot_accuracy(train_acc, val_acc)

### 6. **Visualize Predictions**

In [ ]:
class_names = [
    "Bear", "Bird", "Cat", "Cow", "Deer", "Dog", "Dolphin",
    "Elephant", "Giraffe", "Horse", "Kangaroo", "Lion", "Panda",
    "Tiger", "Zebra"
]

def show_predictions(model, dataloader, num_images=8):
    model.eval()
    images_shown = 0
    plt.figure(figsize=(15, 6))
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            for i in range(images.size(0)):
                if images_shown >= num_images:
                    break
                image_np = images[i].cpu().permute(1, 2, 0).numpy()
                image_np = (image_np - image_np.min()) / (image_np.max() - image_np.min())  # normalize

                plt.subplot(2, int(np.ceil(num_images / 2)), images_shown + 1)
                plt.imshow(image_np)
                plt.title(f"✅ True: {class_names[labels[i]]}\n🔮 Pred: {class_names[preds[i]]}",
                          fontsize=9, color="green" if preds[i] == labels[i] else "red")
                plt.axis("off")
                images_shown += 1
            if images_shown >= num_images:
                break
    plt.tight_layout()
    plt.show()

In [ ]:
# Load test loader
_, _, test_loader = get_dataloaders()

# Show predictions
show_predictions(model, test_loader, num_images=10)

In [ ]:

def show_predictions_one_per_class(model, dataloader, class_names, save_path="../reports/predictions_per_class.png"):
    model.eval()
    shown_classes = set()
    num_classes = len(class_names)
    rows, cols = 3, 5 if num_classes <= 15 else (int(np.ceil(num_classes / 5)), 5)

    plt.figure(figsize=(cols * 3, rows * 3))

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            for i in range(images.size(0)):
                true_cls = labels[i].item()
                if true_cls in shown_classes:
                    continue

                img_np = images[i].cpu().permute(1, 2, 0).numpy()
                img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())

                idx = len(shown_classes)
                plt.subplot(rows, cols, idx + 1)
                plt.imshow(img_np)
                plt.title(f"✅ {class_names[true_cls]}\n🔮 {class_names[preds[i]]}",
                          color="green" if preds[i] == labels[i] else "red", fontsize=9)
                plt.axis("off")
                shown_classes.add(true_cls)

                if len(shown_classes) == num_classes:
                    break
            if len(shown_classes) == num_classes:
                break

    plt.tight_layout()

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=300)
    plt.show()

    print(f"🖼️ Saved prediction preview grid to: {save_path}")
# Show one prediction per class
show_predictions_one_per_class(model, test_loader, class_names)

### 7. **Soothing and Calming Visualizations**

In [ ]:
import plotly.graph_objects as go

def plot_accuracy_plotly(train_acc, val_acc):
    epochs = list(range(1, len(train_acc) + 1))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=epochs, y=train_acc, mode='lines+markers', name='Train Accuracy'))
    fig.add_trace(go.Scatter(x=epochs, y=val_acc, mode='lines+markers', name='Validation Accuracy'))
    fig.update_layout(
        title="📈 Accuracy per Epoch",
        xaxis_title="Epoch",
        yaxis_title="Accuracy",
        template="plotly_white"
    )
    fig.show()
plot_accuracy_plotly(train_acc, val_acc)


In [ ]:
import plotly.express as px

def plot_classwise_metrics(df):
    fig = px.bar(
        df.melt(id_vars="Class", value_vars=["Precision", "Recall", "F1-Score"]),
        x="Class", y="value", color="variable", barmode="group",
        title="🔍 Per-Class Evaluation Metrics"
    )
    fig.update_layout(yaxis_title="Score", template="plotly_white")
    fig.show()
plot_classwise_metrics(results_df)


In [ ]:
import plotly.io as pio

pio.templates["custom_mint"] = go.layout.Template(
    layout=dict(
        font=dict(family="Segoe UI", size=13, color="#333"),
        paper_bgcolor="white",
        plot_bgcolor="white",
        colorway=["#6A5ACD", "#2E8B8B", "#FF6B6B", "#E8C547", "#3CB371"]
    )
)

pio.templates.default = "custom_mint"


#### Accuracy Curves (Train vs. Validation)

In [ ]:
%pip install -U kaleido
import plotly.graph_objects as go
fig = go.Figure(go.Bar(y=[3, 2, 1]))
fig.write_image("test_plot.png")  # if this runs without error, kaleido is working ✅


In [ ]:
import plotly.graph_objects as go


def plot_accuracy_plotly(train_acc, val_acc):
    epochs = list(range(1, len(train_acc) + 1))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=epochs, y=train_acc, mode='lines+markers', name='Train Accuracy'))
    fig.add_trace(go.Scatter(x=epochs, y=val_acc, mode='lines+markers', name='Validation Accuracy'))

    fig.update_layout(
        title="📈 Accuracy over Epochs",
        xaxis_title="Epoch",
        yaxis_title="Accuracy",
        template="plotly_white",
        colorway=["#6A5ACD", "#3CB371"]
    )
    fig.write_image("../reports/plot_accuracy_curve.svg", scale=3)
    
    fig.show()
plot_accuracy_plotly(train_acc, val_acc)


In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix
import numpy as np

def plot_confusion_matrix_plotly(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    cm_percent = np.round(100 * cm / cm.sum(axis=1, keepdims=True), 1)

    fig = ff.create_annotated_heatmap(
        z=cm_percent,
        x=class_names,
        y=class_names,
        colorscale='BuGn',
        showscale=True,
        annotation_text=cm.astype(str),
        hoverinfo="z"
    )
    fig.update_layout(title="🔍 Confusion Matrix (%)", xaxis_title="Predicted", yaxis_title="Actual")
    fig.write_image("../reports/plot_confusion_matrix.png", scale=3)
    fig.show()
plot_confusion_matrix_plotly(all_labels, all_preds, class_names)

In [ ]:
import plotly.express as px

def plot_classwise_metrics(df):  # df = your existing results_df
    fig = px.bar(
        df.melt(id_vars="Class", value_vars=["Precision", "Recall", "F1-Score"]),
        x="Class", y="value", color="variable", barmode="group",
        title="📊 Per-Class Metrics",
        color_discrete_sequence=["#6A5ACD", "#3CB371", "#FF6B6B"]
    )
    fig.update_layout(yaxis_title="Score", template="plotly_white")
    fig.write_image("../reports/plot_classwise_metrics.png", scale=3)
    fig.show()

plot_classwise_metrics(results_df)  # Use your existing results_df

In [ ]:
def plot_class_distribution(df):  # df["Images"] and df["Class"] expected
    fig = px.pie(
        df,
        names="Class",
        values="Images",
        title="🧮 Test Set Class Distribution",
        color_discrete_sequence=px.colors.sequential.Mint
    )
    fig.update_traces(textinfo="percent+label", pull=[0.02]*len(df))
    fig.write_image("../reports/plot_class_distribution_pie.png", scale=3)
    fig.show()
plot_class_distribution(results_df)

In [ ]:
show_predictions_one_per_class(model, test_loader, class_names)


In [ ]:
import matplotlib.pyplot as plt
import os

def plot_misclassified_grid(model, dataloader, class_names, n=12, save_path="../reports/misclassified_grid.png"):
    model.eval()
    errors = []

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            pred = torch.argmax(out, dim=1)

            for img, true_label, pred_label in zip(x, y, pred):
                if true_label != pred_label:
                    errors.append((img.cpu(), true_label.item(), pred_label.item()))
                if len(errors) >= n:
                    break
            if len(errors) >= n:
                break

    if not errors:
        print("🥳 No misclassifications found!")
        return

    rows, cols = (n + 3) // 4, 4
    plt.figure(figsize=(cols * 3, rows * 3))

    for idx, (img, true_idx, pred_idx) in enumerate(errors):
        img_np = img.permute(1, 2, 0).numpy()
        img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())

        plt.subplot(rows, cols, idx + 1)
        plt.imshow(img_np)
        plt.axis("off")
        plt.title(f"True: {class_names[true_idx]}\nPred: {class_names[pred_idx]}", 
                  fontsize=9, 
                  color="red")

    plt.suptitle("❌ Misclassified Samples", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=300)
    plt.show()

    print(f"🖼️ Saved misclassification grid to: {save_path}")
